In [1]:
import pandas as pd
import csv

In [2]:
cali_layoffs = pd.read_csv('california_warn_raw.csv')

In [3]:
cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,Closure Permanent
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,Closure Permanent
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,Closure Permanent
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,Layoff Temporary
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,Layoff Temporary


### Ambiguous Cities

In doing some preliminary exploration of the data, we learned that some cities have been assigned the incorrect county name. The code below finds the cities with more than one county assigned. We use the results produced to go back and clean those records in the WARN Layoffs analysis notebook.

### Clean County Names

In [4]:
cali_layoffs['County'] = cali_layoffs['County'].str.strip()
cali_layoffs['County Orig'] = cali_layoffs['County']
cali_layoffs.loc[~(cali_layoffs['County'].str.endswith('County')),'County'] = cali_layoffs['County'] + " County"
cali_layoffs['County'] = cali_layoffs['County'].str.replace('No County Name County', 'No Name')

In [5]:
cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,Closure Permanent,Los Angeles County
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,Closure Permanent,Los Angeles County
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,Closure Permanent,Orange County
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,Layoff Temporary,San Bernardino County
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,Layoff Temporary,San Diego County


In [6]:
cali_dict = cali_layoffs.to_dict(orient='records')
cali_dict[0:2]

[{'Notice Date': '10/26/2020',
  'Effective Date': '12/28/2020',
  'Received Date': '11/10/2020',
  'Company': 'DAMAC Products LLC',
  'City': 'La Mirada',
  'County': 'Los Angeles County',
  'Employees': 79.0,
  'Layoff/Closure': 'Closure Permanent',
  'County Orig': 'Los Angeles County'},
 {'Notice Date': '10/27/2020',
  'Effective Date': '12/31/2020',
  'Received Date': '11/10/2020',
  'Company': 'Gulfstream',
  'City': 'Long Beach',
  'County': 'Los Angeles County',
  'Employees': 608.0,
  'Layoff/Closure': 'Closure Permanent',
  'County Orig': 'Los Angeles County'}]

In [7]:
cali_layoffs['County'].unique()

array(['Los Angeles County', 'Orange County', 'San Bernardino County',
       'San Diego County', 'Sacramento County', 'Alameda County',
       'Santa Clara County', 'San Francisco County', 'Stanislaus County',
       'Contra Costa County', 'Riverside County', 'Solano County',
       'Ventura County', 'Monterey County', 'Placer County',
       'San Mateo County', 'Santa Barbara County', 'Fresno County',
       'Kings County', 'Kern County', 'Santa Cruz County',
       'Tehama County', 'Mendocino County', 'San Joaquin County',
       'Butte County', 'Sonoma County', 'Napa County', 'Calaveras County',
       'San Luis Obispo County', 'Siskiyou County', 'Yolo County',
       'Yuba County', 'Sutter County', 'Tulare County',
       'El Dorado County', 'Marin County', 'Mariposa County',
       'Imperial County', 'Shasta County', 'Madera County',
       'Del Norte County', 'Mono County', 'Merced County',
       'Humboldt County', 'San Benito County', 'Nevada County',
       'Tuolumne County',

In [8]:
rows_with_no_county = [r for r in cali_dict if r["County"] == "No Name"]
len(rows_with_no_county)

2117

In [9]:
rows_with_no_county[0]

{'Notice Date': '06/30/2016',
 'Effective Date': '08/31/2016',
 'Received Date': '07/01/2016',
 'Company': 'DCS Facility Services',
 'City': 'Antelope',
 'County': 'No Name',
 'Employees': 55.0,
 'Layoff/Closure': 'Closure Permanent',
 'County Orig': 'No County Name'}

In [10]:
cities = [r["City"] for r in rows_with_no_county]

In [11]:
len(cities)

2117

In [12]:
cities = set(cities)

In [13]:
len(cities)

355

In [14]:
rows_with_county = [r for r in cali_dict if r["County"] != "No Name"]

In [15]:
len(rows_with_county)

7390

In [16]:
rows_with_county[334]

{'Notice Date': '06/15/2020',
 'Effective Date': '08/14/2020',
 'Received Date': '09/27/2020',
 'Company': 'Blackhawk Country Club',
 'City': 'Danville',
 'County': 'Contra Costa County',
 'Employees': 30.0,
 'Layoff/Closure': 'Layoff Permanent',
 'County Orig': 'Contra Costa County'}

In [17]:
city_to_counties = {}
for row in rows_with_county:
    city = row["City"]
    county = row["County"]
    if city not in city_to_counties:
        city_to_counties[city] = set()
    city_to_counties[city].add(county)

In [18]:
unambiguous_cities = [pair for pair in city_to_counties.items() if len(pair[1]) == 1]

In [19]:
len(unambiguous_cities)

524

In [20]:
unambiguous_cities = {pair[0]: list(pair[1])[0] for pair in city_to_counties.items() if len(pair[1]) == 1}

In [21]:
for row in rows_with_no_county:
    if row["City"] in unambiguous_cities:
        row["County"] = unambiguous_cities[row["City"]]

In [22]:
rows_with_no_county[:2]

[{'Notice Date': '06/30/2016',
  'Effective Date': '08/31/2016',
  'Received Date': '07/01/2016',
  'Company': 'DCS Facility Services',
  'City': 'Antelope',
  'County': 'No Name',
  'Employees': 55.0,
  'Layoff/Closure': 'Closure Permanent',
  'County Orig': 'No County Name'},
 {'Notice Date': '06/30/2016',
  'Effective Date': '08/31/2016',
  'Received Date': '07/01/2016',
  'Company': 'DCS Facility Services',
  'City': 'Bakersfield',
  'County': 'Kern County',
  'Employees': 22.0,
  'Layoff/Closure': 'Closure Permanent',
  'County Orig': 'No County Name'}]

In [23]:
ambiguous_cities = [pair for pair in city_to_counties.items() if len(pair[1]) > 1]

In [24]:
ambiguous_cities

[('Sacramento', {'Del Norte County', 'Sacramento County'}),
 ('San Francisco', {'San Francisco County', 'San Mateo County'}),
 ('Hayward', {'Alameda County', 'Contra Costa County'}),
 ('Indian Wells', {'Riverside County', 'San Benito County'}),
 ('Cerritos', {'Los Angeles County', 'Orange County'}),
 ('Vista', {'San Diego County', 'San Francisco County'}),
 ('Roseville', {'Placer County', 'Sacramento County'}),
 ('City of Industry', {'Los Angeles County', 'Orange County'}),
 ('Del Mar', {'Orange County', 'San Diego County'}),
 ('Walnut Creek', {'Alameda County', 'Contra Costa County'}),
 ('Los Gatos', {'Santa Clara County', 'Santa Cruz County'}),
 ('Rancho', {'Los Angeles County', 'San Bernardino County'})]

### Manual research narrowed down which cities actually belonged to which counties
resulting in our updated `ambiguous_cities` dictionary. `Rancho` is the only city on the list where the two counties actually have cities with the same name. It will be left off the list.

In [25]:
dict_cities = {
    'Sacramento':'Sacramento County',
    'San Francisco':'San Francisco County',
    'Hayward':'Alameda County',
    'Indian Wells':'Riverside County',
    'Cerritos':'Los Angeles County',
    'Vista':'San Diego County',
    'Roseville':'Placer County',
    'City of Industry':'Los Angeles County',
    'Del Mar':'San Diego County',
    'Walnut Creek':'Contra Costa County',
    'Los Gatos':'Santa Clara County'
}

In [26]:
cali_layoffs.to_csv('california_warn_raw_2.csv')

### Ambiguous Cities

The cities listed above are the only cities that need to be changed for our code before continuing our analysis. In our analysis notebook, we will find these specific records and update them accordingly.